# データベース

- データを一元管理。
- さまざまなデータを関連付けて格納。
- 重複データを認めないなど制約をつけてデータを格納。
- データの一貫性（整合性）を確保できる。
- 複数人でデータを共有できる。
- データへの同時アクセスを処理できる。
- 大量のデータでも、少しずつ読み出したり、並べ替えたりできる。

## SQLite

In [1]:
import sqlite3

# sqliteのデータベースに接続 --- (※1)
dbpath = "file/test.sqlite"
conn = sqlite3.connect(dbpath)

# テーブルを作成し、データを挿入する --- (※2)
cur = conn.cursor()   # データベースを操作するカーソルを取得。
cur.executescript("""
/* itemsテーブルが既にあれば削除する */
DROP TABLE IF EXISTS items;
/* テーブルの作成 */
CREATE TABLE items(
    item_id INTEGER PRIMARY KEY,
    name TEXT UNIQUE,
    price INTEGER
);
/* データを挿入  */
INSERT INTO items(name, price)VALUES('Apple', 800);
INSERT INTO items(name, price)VALUES('Orange', 780);
INSERT INTO items(name, price)VALUES('Banana', 430);
""")
# 上記の操作をデータベースに反映させる --- (※3)
conn.commit()

# データを抽出する --- (※4)
cur = conn.cursor()
cur.execute("SELECT item_id,name,price FROM items")
item_list = cur.fetchall()   # 全て取得。
# 一行ずつ表示
for it in item_list:
    print(it)

(1, 'Apple', 800)
(2, 'Orange', 780)
(3, 'Banana', 430)


In [2]:
import sqlite3

# データベースに接続 --- (※1)
filepath = "file/test2.sqlite"
conn = sqlite3.connect(filepath)

# テーブルを作成 --- (※2)
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS items") 
cur.execute(""" CREATE TABLE items (
    item_id INTEGER PRIMARY KEY,
    name    TEXT,
    price   INTEGER)""")
conn.commit()

# 単発でデータを挿入 --- (※3)
cur = conn.cursor()
cur.execute(
    "INSERT INTO items (name,price) VALUES (?,?)",   # 挿入する値を「？」を使って表現。
    ("Orange", 520))
conn.commit()

# 連続でデータを挿入 --- (※4)
cur = conn.cursor()
data = [("Mango",770), ("Kiwi",400), ("Grape",800),
    ("Peach",940),("Persimmon",700),("Banana", 400)]
cur.executemany(
    "INSERT INTO items(name,price) VALUES (?,?)",
    data)
conn.commit()

# 400-700円のデータを抽出して表示 --- (※5)
cur = conn.cursor()
price_range = (400, 700)
cur.execute(
    "SELECT * FROM items WHERE price>=? AND price<=?",
    price_range)
fr_list = cur.fetchall()
for fr in fr_list:
    print(fr)

(1, 'Orange', 520)
(3, 'Kiwi', 400)
(6, 'Persimmon', 700)
(7, 'Banana', 400)


## MySQL

In [3]:
# # ライブラリのインポート --- (※1)
# import MySQLdb

# # MySQLに接続する --- (※2)
# conn = MySQLdb.connect(
#     user='root',
#     passwd='test-password',
#     host='localhost',
#     db='test')

# # カーソルを取得する --- (※3)
# cur = conn.cursor()

# # テーブルを作成する --- (※4)
# cur.execute('DROP TABLE IF EXISTS items')
# cur.execute('''
#     CREATE TABLE items (
#         item_id INTEGER PRIMARY KEY AUTO_INCREMENT,
#         name TEXT,
#         price INTEGER
#     )
#     ''')

# # データを挿入する --- (※5)
# data = [('Banana', 300),('Mango', 640), ('Kiwi', 280)]
# for i in data:
#     cur.execute("INSERT INTO items(name,price) VALUES(%s,%s)", i)

# # データを抽出する --- (※6)
# cur.execute("SELECT * FROM items")
# for row in cur.fetchall():
#     print(row)

## TinyDB

In [4]:
# TinyDBを使うためライブラリのインポート
from tinydb import TinyDB, Query

# データベースに接続 --- (※1)
filepath = "file/test-tynydb.json"
db = TinyDB(filepath)

# 既存のデータがあれば破棄 --- (※2)
# db.purge_table('fruits')

# テーブルを得る --- (※3)
table = db.table('fruits')

# データをデータベースに挿入 --- (※4)
table.insert( {'name': 'Banana', 'price': 600} )
table.insert( {'name': 'Orange', 'price': 1200} )
table.insert( {'name': 'Mango', 'price': 840} )

# 全データを抽出して表示 --- (※5)
print(table.all())

# 特定のデータを抽出して表示
# Orangeを検索 --- (※6)
Item = Query()
res = table.search(Item.name == 'Orange')
print('Orange is ', res[0]['price'])

# 値段が800円以上のものを抽出 --- (※7)
print("800円以上のもの:")
res = table.search(Item.price >= 800)
for it in res:
    print("-", it['name'])

[{'name': 'Banana', 'price': 600}, {'name': 'Orange', 'price': 1200}, {'name': 'Mango', 'price': 840}, {'name': 'Banana', 'price': 600}, {'name': 'Orange', 'price': 1200}, {'name': 'Mango', 'price': 840}]
Orange is  1200
800円以上のもの:
- Orange
- Mango
- Orange
- Mango
